In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df.APPLICATION_TYPE.value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_type_counts = application_df.CLASSIFICATION.value_counts()
classification_type_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C4200        1
C2561        1
C2600        1
C1248        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_type_counts[classification_type_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

#Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Three Hidden Layer Attempt

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1= 80
hidden_nodes_layer2= 30
hidden_nodes_layer3 = 12

nn = tf.keras.models.Sequential()

#First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

#Output Layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 12)                372       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 13        
Total params: 6,335
Trainable params: 6,335
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 65us/sample - loss: 14308.9860 - accuracy: 0.4908
Epoch 2/100
25724/25724 [==============================] - 1s 37us/sample - loss: 37193.3055 - accuracy: 0.4865
Epoch 3/100
25724/25724 [==============================] - 1s 41us/sample - loss: 17063.7766 - accuracy: 0.4944
Epoch 4/100
25724/25724 [==============================] - 1s 35us/sample - loss: 26863.0821 - accuracy: 0.4998
Epoch 5/100
25724/25724 [==============================] - 1s 45us/sample - loss: 11264.3074 - accuracy: 0.5058
Epoch 6/100
25724/25724 [==============================] - 1s 41us/sample - loss: 7549.9089 - accuracy: 0.5094
Epoch 7/100
25724/25724 [==============================] - 1s 40us/sample - loss: 2470.3779 - accuracy: 0.4836
Epoch 8/100
25724/25724 [==============================] - 1s 37us/sample - loss: 1891.0520 - accuracy: 0.4969
Epoch 9/100
25724/25724 [==============================] - 1s 38us/sample - loss: 19

25724/25724 [==============================] - 1s 36us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 76/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 77/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 78/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 79/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 80/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 81/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 82/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 83/100
25724/25724 [==============================] - 1s 40us/sample - loss: 0.6911 - accuracy: 0.5321
Epoch 84/100
25724/25724 [======

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.7001 - accuracy: 0.5329
Loss: 0.7000641994365102, Accuracy: 0.5329446196556091


# Additional neurons added to two layers

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1= 80
hidden_nodes_layer2= 55

nn = tf.keras.models.Sequential()

#First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#Output Layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 80)                3520      
_________________________________________________________________
dense_5 (Dense)              (None, 55)                4455      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 56        
Total params: 8,031
Trainable params: 8,031
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 1s 57us/sample - loss: 14319.1072 - accuracy: 0.4934
Epoch 2/100
25724/25724 [==============================] - 1s 38us/sample - loss: 36740.0342 - accuracy: 0.4957
Epoch 3/100
25724/25724 [==============================] - 1s 39us/sample - loss: 25475.5829 - accuracy: 0.5042
Epoch 4/100
25724/25724 [==============================] - 1s 38us/sample - loss: 29279.4170 - accuracy: 0.4888
Epoch 5/100
25724/25724 [==============================] - 1s 38us/sample - loss: 25905.4612 - accuracy: 0.5038
Epoch 6/100
25724/25724 [==============================] - 1s 38us/sample - loss: 6676.2214 - accuracy: 0.5140
Epoch 7/100
25724/25724 [==============================] - 1s 37us/sample - loss: 7809.6186 - accuracy: 0.5039
Epoch 8/100
25724/25724 [==============================] - 1s 38us/sample - loss: 18097.0635 - accuracy: 0.5060
Epoch 9/100
25724/25724 [==============================] - 1s 38us/sample - loss: 7

25724/25724 [==============================] - 1s 36us/sample - loss: 0.6913 - accuracy: 0.5321
Epoch 76/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6913 - accuracy: 0.5322
Epoch 77/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6913 - accuracy: 0.5313
Epoch 78/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6914 - accuracy: 0.5321
Epoch 79/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6914 - accuracy: 0.5321
Epoch 80/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6913 - accuracy: 0.5309
Epoch 81/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6914 - accuracy: 0.5321
Epoch 82/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6913 - accuracy: 0.5314
Epoch 83/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6912 - accuracy: 0.5321
Epoch 84/100
25724/25724 [======

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6619 - accuracy: 0.6024
Loss: 0.6618887167908359, Accuracy: 0.6024489998817444


# Activation function of hidden layer is changed

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1= 80
hidden_nodes_layer2= 30

nn = tf.keras.models.Sequential()

#First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))


#Output Layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 80)                3520      
_________________________________________________________________
dense_8 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.6919 - accuracy: 0.5307
Epoch 2/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6913 - accuracy: 0.5321
Epoch 3/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6913 - accuracy: 0.5312
Epoch 4/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6914 - accuracy: 0.5321
Epoch 5/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.6915 - accuracy: 0.5321
Epoch 6/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6913 - accuracy: 0.5311
Epoch 7/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6914 - accuracy: 0.5303
Epoch 8/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6915 - accuracy: 0.5321
Epoch 9/100
25724/25724 [==============================] - 1s 37us/sample - loss: 0.6914 - accuracy: 0.5298
Epoch

25724/25724 [==============================] - 1s 35us/sample - loss: 0.6914 - accuracy: 0.5321
Epoch 76/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6914 - accuracy: 0.5321
Epoch 77/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6915 - accuracy: 0.5321
Epoch 78/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6914 - accuracy: 0.5304
Epoch 79/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6914 - accuracy: 0.5321
Epoch 80/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6913 - accuracy: 0.5313
Epoch 81/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6914 - accuracy: 0.5313
Epoch 82/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6914 - accuracy: 0.5314
Epoch 83/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6913 - accuracy: 0.5321
Epoch 84/100
25724/25724 [======

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.7385 - accuracy: 0.4667
Loss: 0.7384962310248839, Accuracy: 0.4667055308818817


In [25]:
#Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [26]:
#Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

In [27]:
#Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

#Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
   32/25724 [..............................] - ETA: 6:38 - loss: 0.7269 - accuracy: 0.5000
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1024/25724 [>.............................] - ETA: 14s - loss: 0.6805 - accuracy: 0.5986 
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2368/25724 [=>............................] - ETA: 7s - loss: 0.6511 - accuracy: 0.6529
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 3904/25724 [===>..........................] - ETA: 5s - loss: 0.6222 - accuracy: 0.6865
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 5088/25724 [====>.........................] - ETA: 4s - loss: 0.6115 - accuracy: 0.6965
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 5920/25724 [=====>........................] - ETA: 3s - loss: 0.6100 - accuracy: 0.6949
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 7072/25724 [=======>......................] - ETA: 3s - loss: 0.6032 - accuracy: 0.6

 3840/25724 [===>..........................] - ETA: 2s - loss: 0.5508 - accuracy: 0.7331
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 5664/25724 [=====>........................] - ETA: 2s - loss: 0.5493 - accuracy: 0.7323
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 5888/25724 [=====>........................] - ETA: 2s - loss: 0.5511 - accuracy: 0.7303
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 7872/25724 [========>.....................] - ETA: 1s - loss: 0.5525 - accuracy: 0.7308
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 7968/25724 [========>.....................] - ETA: 1s - loss: 0.5529 - accuracy: 0.7307
Epoch 00003: saving model to checkpoints/weights.03.hdf5
 8960/25724 [=========>....................] - ETA: 1s - loss: 0.5524 - accuracy: 0.7305
Epoch 00003: saving model to checkpoints/weights.03.hdf5
10912/25724 [===========>..................] - ETA: 1s - loss: 0.5540 - accuracy: 0.7286
Epoch 00003: saving model to checkp

 9120/25724 [=========>....................] - ETA: 2s - loss: 0.5438 - accuracy: 0.7361
Epoch 00005: saving model to checkpoints/weights.05.hdf5
10496/25724 [===========>..................] - ETA: 1s - loss: 0.5463 - accuracy: 0.7358
Epoch 00005: saving model to checkpoints/weights.05.hdf5
11680/25724 [============>.................] - ETA: 1s - loss: 0.5462 - accuracy: 0.7356
Epoch 00005: saving model to checkpoints/weights.05.hdf5
11776/25724 [============>.................] - ETA: 1s - loss: 0.5460 - accuracy: 0.7360
Epoch 00005: saving model to checkpoints/weights.05.hdf5
12800/25724 [=============>................] - ETA: 1s - loss: 0.5462 - accuracy: 0.7355
Epoch 00005: saving model to checkpoints/weights.05.hdf5
14400/25724 [===============>..............] - ETA: 1s - loss: 0.5468 - accuracy: 0.7349
Epoch 00005: saving model to checkpoints/weights.05.hdf5
15392/25724 [================>.............] - ETA: 1s - loss: 0.5476 - accuracy: 0.7340
Epoch 00005: saving model to checkp

13568/25724 [==============>...............] - ETA: 1s - loss: 0.5471 - accuracy: 0.7356
Epoch 00007: saving model to checkpoints/weights.07.hdf5
15360/25724 [================>.............] - ETA: 1s - loss: 0.5465 - accuracy: 0.7357
Epoch 00007: saving model to checkpoints/weights.07.hdf5
16160/25724 [=================>............] - ETA: 1s - loss: 0.5481 - accuracy: 0.7350
Epoch 00007: saving model to checkpoints/weights.07.hdf5
16992/25724 [==================>...........] - ETA: 0s - loss: 0.5462 - accuracy: 0.7363
Epoch 00007: saving model to checkpoints/weights.07.hdf5
17696/25724 [===================>..........] - ETA: 0s - loss: 0.5462 - accuracy: 0.7358
Epoch 00007: saving model to checkpoints/weights.07.hdf5
18688/25724 [====================>.........] - ETA: 0s - loss: 0.5471 - accuracy: 0.7354
Epoch 00007: saving model to checkpoints/weights.07.hdf5
19712/25724 [=====================>........] - ETA: 0s - loss: 0.5474 - accuracy: 0.7353
Epoch 00007: saving model to checkp

19072/25724 [=====================>........] - ETA: 0s - loss: 0.5464 - accuracy: 0.7381
Epoch 00009: saving model to checkpoints/weights.09.hdf5
19456/25724 [=====================>........] - ETA: 0s - loss: 0.5466 - accuracy: 0.7373
Epoch 00009: saving model to checkpoints/weights.09.hdf5
20480/25724 [======================>.......] - ETA: 0s - loss: 0.5469 - accuracy: 0.7368
Epoch 00009: saving model to checkpoints/weights.09.hdf5
21504/25724 [========================>.....] - ETA: 0s - loss: 0.5477 - accuracy: 0.7356
Epoch 00009: saving model to checkpoints/weights.09.hdf5
22528/25724 [=========================>....] - ETA: 0s - loss: 0.5479 - accuracy: 0.7352
Epoch 00009: saving model to checkpoints/weights.09.hdf5
23552/25724 [==========================>...] - ETA: 0s - loss: 0.5482 - accuracy: 0.7347
Epoch 00009: saving model to checkpoints/weights.09.hdf5
24576/25724 [===========================>..] - ETA: 0s - loss: 0.5479 - accuracy: 0.7346
Epoch 00009: saving model to checkp

...

 8256/25724 [========>.....................] - ETA: 2s - loss: 0.5317 - accuracy: 0.7479
Epoch 00092: saving model to checkpoints/weights.92.hdf5
 8832/25724 [=========>....................] - ETA: 2s - loss: 0.5308 - accuracy: 0.7490
Epoch 00092: saving model to checkpoints/weights.92.hdf5
10592/25724 [===========>..................] - ETA: 1s - loss: 0.5299 - accuracy: 0.74 - ETA: 1s - loss: 0.5299 - accuracy: 0.7488
Epoch 00092: saving model to checkpoints/weights.92.hdf5
10912/25724 [===========>..................] - ETA: 1s - loss: 0.5302 - accuracy: 0.7484
Epoch 00092: saving model to checkpoints/weights.92.hdf5
11904/25724 [============>.................] - ETA: 1s - loss: 0.5308 - accuracy: 0.7470
Epoch 00092: saving model to checkpoints/weights.92.hdf5
13888/25724 [===============>..............] - ETA: 1s - loss: 0.5320 - accuracy: 0.7454
Epoch 00092: saving model to checkpoints/weights.92.hdf5
14944/25724 [================>.............] - ETA: 1s - loss: 0.5319 - accuracy: 

12416/25724 [=============>................] - ETA: 1s - loss: 0.5292 - accuracy: 0.7469
Epoch 00094: saving model to checkpoints/weights.94.hdf5
12672/25724 [=============>................] - ETA: 1s - loss: 0.5286 - accuracy: 0.7470
Epoch 00094: saving model to checkpoints/weights.94.hdf5
13696/25724 [==============>...............] - ETA: 1s - loss: 0.5283 - accuracy: 0.7477
Epoch 00094: saving model to checkpoints/weights.94.hdf5
14720/25724 [================>.............] - ETA: 1s - loss: 0.5300 - accuracy: 0.7457
Epoch 00094: saving model to checkpoints/weights.94.hdf5
16224/25724 [=================>............] - ETA: 0s - loss: 0.5300 - accuracy: 0.7450
Epoch 00094: saving model to checkpoints/weights.94.hdf5
16768/25724 [==================>...........] - ETA: 0s - loss: 0.5305 - accuracy: 0.7445
Epoch 00094: saving model to checkpoints/weights.94.hdf5
17792/25724 [===================>..........] - ETA: 0s - loss: 0.5316 - accuracy: 0.7439
Epoch 00094: saving model to checkp

15488/25724 [=================>............] - ETA: 1s - loss: 0.5314 - accuracy: 0.7432
Epoch 00096: saving model to checkpoints/weights.96.hdf5
16896/25724 [==================>...........] - ETA: 0s - loss: 0.5324 - accuracy: 0.7428
Epoch 00096: saving model to checkpoints/weights.96.hdf5
17536/25724 [===================>..........] - ETA: 0s - loss: 0.5332 - accuracy: 0.7429
Epoch 00096: saving model to checkpoints/weights.96.hdf5
18560/25724 [====================>.........] - ETA: 0s - loss: 0.5342 - accuracy: 0.7426
Epoch 00096: saving model to checkpoints/weights.96.hdf5
19584/25724 [=====================>........] - ETA: 0s - loss: 0.5353 - accuracy: 0.7416
Epoch 00096: saving model to checkpoints/weights.96.hdf5
20608/25724 [=======================>......] - ETA: 0s - loss: 0.5355 - accuracy: 0.7416
Epoch 00096: saving model to checkpoints/weights.96.hdf5
21632/25724 [========================>.....] - ETA: 0s - loss: 0.5349 - accuracy: 0.7419
Epoch 00096: saving model to checkp

20736/25724 [=======================>......] - ETA: 0s - loss: 0.5310 - accuracy: 0.7447
Epoch 00098: saving model to checkpoints/weights.98.hdf5
22336/25724 [=========================>....] - ETA: 0s - loss: 0.5324 - accuracy: 0.7427
Epoch 00098: saving model to checkpoints/weights.98.hdf5
22848/25724 [=========================>....] - ETA: 0s - loss: 0.5326 - accuracy: 0.7427
Epoch 00098: saving model to checkpoints/weights.98.hdf5
24384/25724 [===========================>..] - ETA: 0s - loss: 0.5334 - accuracy: 0.7420
Epoch 00098: saving model to checkpoints/weights.98.hdf5
24544/25724 [===========================>..] - ETA: 0s - loss: 0.5338 - accuracy: 0.7415
Epoch 00098: saving model to checkpoints/weights.98.hdf5
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5345 - accuracy: 0.7414
Epoch 99/100
   32/25724 [..............................] - ETA: 3s - loss: 0.5379 - accuracy: 0.7188
Epoch 00099: saving model to checkpoints/weights.99.hdf5
 1568/25724 [>

24192/25724 [===========================>..] - ETA: 0s - loss: 0.5354 - accuracy: 0.7398
Epoch 00100: saving model to checkpoints/weights.100.hdf5
25724/25724 [==============================] - 3s 127us/sample - loss: 0.5347 - accuracy: 0.7411
8575/8575 - 0s - loss: 0.5533 - accuracy: 0.7248
Loss: 0.5533263656766352, Accuracy: 0.724781334400177


In [28]:
#Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")